<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Dashboard

For this notebook, you are building a prototype dashboard. The specific user persona we have in mind for you to design around is a non-technical instructor who is relatively new to teaching online and has moderate to low data literacy skills. 

The instructor, as we have mentioned in previous Notebooks, is looking to run a report about 25% of the way through the course in order to identify who he or she should do a formal check-in with.
 
To help you in building your dashboard, this notebook provides a brief introduction to the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) for building a [Plotly Dash](https://github.com/plotly/dash) app within Jupyter environments. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

You can also use other packages like `seaborn`. **The only caveat is that the resulting visualization needs to be interactive.**

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/) as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by entering your email.

In [1]:
# !pip install jupyter-dash  ## should already be installed

import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
#JupyterDash.infer_jupyter_proxy_config()

In [2]:
lr = pd.read_csv('assets/learning_resources.csv')
qt = pd.read_csv('assets/quizzes_tests.csv')
si = pd.read_csv('assets/student_info_pred.csv')
df = pd.read_csv('assets/country_indicators.csv')

# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 



## Step 1. The Dash components (i.e. layouts)

In [3]:
score = qt.groupby('assignment_name')['score'].agg({np.mean,np.median, np.std}).reset_index(drop = False)

In [4]:
app = JupyterDash(__name__)

# Create server variable
server = app.server

# Create a unique list of code_module
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
    
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id = 'graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id = 'crossfilter-xaxis-column',
                options = [{'label': i, 'value': i} for i in assess_list],
                value ='Quiz 1' # the default assignment
            )])

# Run the app
app.run_server(mode = "inline", port = 8100)

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [5]:
# Create a unique list of code_module
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in assess_list],
                value='Quiz 1' # the default
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    filtered_score = score[(score.assignment_name == assignment_name)]
    figure = px.bar(filtered_score, x = 'assignment_name', y = 'mean')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

In [6]:
# Create a unique list of assignment_name
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in assess_list],
                    value=assess_list # Default values contain all assignment_name
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(values):
    figure = px.scatter(
                     score[score["assignment_name"].isin(values)],
                     x="mean",
                     y="std",
                     color="assignment_name",
                     hover_name="assignment_name")
    return figure # You must return all the output(s) in step 
    
# Run the app
app.run_server(mode="inline", port = 8102)

# Building Your Dashboard (40 points)

Your final product will need to have the following capabilities:

1. (10 points) View multiple students' predicted probabilities of failing the course
2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test
3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource
4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

This notebook is broken out into four sections, three to build individual components and one to integrate them. The final Integrated Dashboard cell will be graded and must include all three specified capabilities. If it does not, points will be awarded based on progress made in each Component cell.

There is a new column in `student_info.csv`, `fail_pred,` that represents the predicted probability of failure for each student from a baseline model.

**Note:** All quizzes, tests, and resources should only include information from on or before day 60 in the course.

### Dictionary
- **student_info.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **gender** = character; M = "male", F = "female"
    - **highest_education** = character; “Some Graduate”, “Some Higher Education”, “High School + Advanced Placement”, “High School”, “No Formal Quals” (Categories ordered from highest documented education level attained to lowest documented education level attained)
    - **disability** = character; Y = "yes", N = "no"
    - **final_result** = character; "Fail", "Pass"
    - **fail_pred** = numeric; predicted probability from sample model


- **quizzes_tests.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **assignment_name** = character; name of graded assignment (Quiz 1-7, Test 1-6, Final Exam)
    - **due_date** = numeric; date assignment was due (indexed as count in days from start of course, i.e., day 0)
    - **weight** = numeric; weight multiplied by score when generating final grade (weight * score / 100)
    - **date_submitted** = numeric; date student submitted assigned (indexed as count in days from start of course, i.e., day 0, NaN means students did not submit assignment)
    - **score** = numeric; score student earned on assignment (0 means students did not submit assignment)


- **learning_resources.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **activity_type** = character; overarching label for learning activity students can access (“course_homepage”, “course_page”, “forum”, ‘resource”, “wiki”)
    - **activity_id** = numeric; unique identifier for specific learning activity student accessed within overacting `activity_type`
    - **date** = numeric; date student accessed specific `acitivity_id`  (indexed as count in days from start of course, i.e., day 0)
    - **sum_click** = numeric; count of clicks for `activity_id` on date

## Component 1
1. (10 points) View multiple students' predicted probabilities of failing the course

In [7]:
# YOUR CODE HERE
si.columns
#raise NotImplementedError()

Index(['pred_fail', 'id_student', 'gender', 'highest_education', 'disability',
       'final_result'],
      dtype='object')

In [8]:
si["id_student"] = si["id_student"].astype(str)

In [9]:
app = JupyterDash("new")

# Create server variable
server = app.server

# Create a unique list of assignment_name
assess_list = si['id_student'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Probability of Passing, multiple students"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in assess_list],
                    value=[assess_list[0]] # Default values contain all assignment_name
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(values):
    filtered_students = si[si["id_student"].isin(values)]
    figure = px.bar(filtered_students, y = 'pred_fail', x = 'id_student')
    return figure # You must return all the output(s) in step 
    
# Run the app
app.run_server(mode="inline", port = 8100)

## Component 2

2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [10]:
# YOUR CODE HERE
qt_avg = qt.merge(qt.groupby("assignment_name")["score"].mean().reset_index().rename(columns={"score":"class_average"}), on="assignment_name").rename(columns={"score":"student_score"})
qt_avg["status"] = "on time"
qt_avg.loc[qt_avg["due_date"] < qt_avg["date_submitted"], "status"] = "late"

qt_avg.loc[qt_avg["date_submitted"].isna(), "status"] = "incomplete"

qt_avg.melt(id_vars=["id_student", "assignment_name"], value_vars=["student_score", "class_average"], var_name="Score Group", value_name="Score")
#raise NotImplementedError()

,id_student,assignment_name,Score Group,Score
0,41060,Quiz 1,student_score,77.000000
1,45664,Quiz 1,student_score,47.000000
2,52014,Quiz 1,student_score,53.000000
3,53488,Quiz 1,student_score,93.000000
4,60135,Quiz 1,student_score,57.000000
...,...,...,...,...
22759,2678338,Final Exam,class_average,41.728167
22760,2683836,Final Exam,class_average,41.728167
22761,2689536,Final Exam,class_average,41.728167
22762,2693243,Final Exam,class_average,41.728167


In [11]:
app = JupyterDash("new")

# Create server variable
server = app.server

qt_avg = qt.merge(qt.groupby("assignment_name")["score"].mean().reset_index().rename(columns={"score":"class_average"}), on="assignment_name").rename(columns={"score":"student_score"})
qt_avg["status"] = "on time"
qt_avg.loc[qt_avg["due_date"] < qt_avg["date_submitted"], "status"] = "late"

qt_avg.loc[qt_avg["date_submitted"].isna(), "status"] = "incomplete"

qt_avg = qt_avg.melt(id_vars=["id_student", "assignment_name", "due_date"], value_vars=["student_score", "class_average"], var_name="Score Group", value_name="Score")

qt_avg = qt_avg[qt_avg["due_date"] <= 60].copy()

# Create a unique list of assignment_name
assess_list = qt['id_student'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Student's Performance & Class Average"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='dropdown',
                options=[{'label': i, 'value': i} for i in assess_list],
                value= assess_list[0]# the default
            )])


# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('dropdown', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(values):
    figure = px.line(
                     qt_avg[qt_avg["id_student"]==values].sort_values("due_date"),
                     x="assignment_name",
                     y="Score",
                     color="Score Group",
                     hover_name="assignment_name")
    return figure # You must return all the output(s) in step 
    
# Run the app
app.run_server(mode="inline", port = 8102)

## Component 3

3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [12]:
# YOUR CODE HERE
lr[lr.duplicated(subset=["id_student", "date", "lr_id"], keep=False)].sort_values(["id_student", "date", "lr_id"])
#raise NotImplementedError()

KeyError: Index(['lr_id'], dtype='object')

In [13]:
lr["lr_id"] = lr["activity_type"] + " - " + lr["activity_id"].astype(str)
lr_daily = lr.groupby(["id_student", "date", "lr_id"] )["sum_click"].sum().reset_index()
class_avg = lr_daily.groupby(["date", "lr_id"])["sum_click"].mean().reset_index()

lr_daily = lr_daily.merge(class_avg, on=["date", "lr_id"])
lr_daily.columns = ["id_student", "date", "lr_id", "student_usage (clicks)", "average_usage (clicks)"]
lr_daily = lr_daily.melt(id_vars = ["id_student", "date", "lr_id"], value_vars = ["student_usage (clicks)", "average_usage (clicks)"], var_name="Usage Group", value_name="Clicks")
lr_daily

,id_student,date,lr_id,Usage Group,Clicks
0,41060,0,course_homepage - 1,student_usage (clicks),5.0
1,45664,0,course_homepage - 1,student_usage (clicks),32.0
2,53488,0,course_homepage - 1,student_usage (clicks),70.0
3,60135,0,course_homepage - 1,student_usage (clicks),131.0
4,66579,0,course_homepage - 1,student_usage (clicks),15.0
...,...,...,...,...,...
751131,2694933,37,resource - 107,average_usage (clicks),1.0
751132,2694933,108,forum - 10,average_usage (clicks),2.0
751133,2694933,150,wiki - 9,average_usage (clicks),3.0
751134,2694933,199,resource - 104,average_usage (clicks),1.0


In [14]:
app = JupyterDash("new2")

# Create server variable
server = app.server

lr["lr_id"] = lr["activity_type"] + " - " + lr["activity_id"].astype(str)
lr_daily = lr.groupby(["id_student", "date", "lr_id"] )["sum_click"].sum().reset_index()
class_avg = lr_daily.groupby(["date", "lr_id"])["sum_click"].mean().reset_index()

lr_daily = lr_daily.merge(class_avg, on=["date", "lr_id"])
lr_daily.columns = ["id_student", "date", "lr_id", "student_usage (clicks)", "average_usage (clicks)"]
lr_daily = lr_daily.melt(id_vars = ["id_student", "date", "lr_id"], value_vars = ["student_usage (clicks)", "average_usage (clicks)"], var_name="Usage Group", value_name="Clicks")
lr_daily = lr_daily[lr_daily["date"] <= 60].copy()
lr_daily

# Create a unique list of assignment_name
student_list = lr['id_student'].unique()
lr_list = lr['lr_id'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Student's LR Usage & Class Average"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='student_dropdown',
                options=[{'label': i, 'value': i} for i in student_list],
                value= student_list[0]# the default
            ),

            dcc.Dropdown(
                id='lr_dropdown',
                options=[{'label': i, 'value': i} for i in lr_list],
                value= lr_list[0]# the default
            )
]
    )


# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('student_dropdown', 'value'),
    dash.dependencies.Input('lr_dropdown', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(studnet_id, lr_id):
    figure = px.line(
                     lr_daily[(lr_daily["id_student"]==studnet_id) & (lr_daily["lr_id"]==lr_id)].sort_values("date"),
                     x="date",
                     y="Clicks",
                     color="Usage Group",
                     hover_name="lr_id")
    return figure # You must return all the output(s) in step 
    
# Run the app
app.run_server(mode="inline", port = 8102)

---

## Integrated Dashboard

4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

In [15]:
# YOUR CODE HERE
app = JupyterDash("new2")

# Create server variable
server = app.server

lr["lr_id"] = lr["activity_type"] + " - " + lr["activity_id"].astype(str)
lr_daily = lr.groupby(["id_student", "date", "lr_id"] )["sum_click"].sum().reset_index()
class_avg = lr_daily.groupby(["date", "lr_id"])["sum_click"].mean().reset_index()

lr_daily = lr_daily.merge(class_avg, on=["date", "lr_id"])
lr_daily.columns = ["id_student", "date", "lr_id", "student_usage (clicks)", "average_usage (clicks)"]
lr_daily = lr_daily.melt(id_vars = ["id_student", "date", "lr_id"], value_vars = ["student_usage (clicks)", "average_usage (clicks)"], var_name="Usage Group", value_name="Clicks")
lr_daily = lr_daily[lr_daily["date"] <= 60].copy()
lr_daily

# Create a unique list of assignment_name
student_list = lr['id_student'].unique()
lr_list = lr['lr_id'].unique()

qt_avg = qt.merge(qt.groupby("assignment_name")["score"].mean().reset_index().rename(columns={"score":"class_average"}), on="assignment_name").rename(columns={"score":"student_score"})
qt_avg["status"] = "on time"
qt_avg.loc[qt_avg["due_date"] < qt_avg["date_submitted"], "status"] = "late"

qt_avg.loc[qt_avg["date_submitted"].isna(), "status"] = "incomplete"

qt_avg = qt_avg.melt(id_vars=["id_student", "assignment_name", "due_date"], value_vars=["student_score", "class_average"], var_name="Score Group", value_name="Score")

qt_avg = qt_avg[qt_avg["due_date"] <= 60].copy()


app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("Student's LR Usage & Class Average"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='student_dropdown',
                options=[{'label': i, 'value': i} for i in student_list],
                value= student_list[0]# the default
            ),

            dcc.Dropdown(
                id='lr_dropdown',
                options=[{'label': i, 'value': i} for i in lr_list],
                value= lr_list[0]# the default
            ),
     # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in student_list],
                    value=[student_list[0]] # Default values contain all assignment_name
            )
]
    )



@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value'),
     dash.dependencies.Input('student_dropdown', 'value'),
    dash.dependencies.Input('lr_dropdown', 'value')])

def update_figure(values, studnet_id, lr_id):
    filtered_students = si[si["id_student"].isin(values)]
    figure1 = px.bar(filtered_students, y = 'pred_fail', x = 'id_student')
        
    figure2 = px.line(
                 qt_avg[qt_avg["id_student"]==studnet_id].sort_values("due_date"),
                 x="assignment_name",
                 y="Score",
                 color="Score Group",
                 hover_name="assignment_name")
    
    figure3 = px.line(
                     lr_daily[(lr_daily["id_student"]==studnet_id) & (lr_daily["lr_id"]==lr_id)].sort_values("date"),
                     x="date",
                     y="Clicks",
                     color="Usage Group",
                     hover_name="lr_id")

    return  
    
# Run the app
app.run_server(mode="inline", port = 8102)

#raise NotImplementedError()

## 2. Interpret (10 points)

After developing your integrated dashboard, provide your thoughs on the following questions:

2.1 How well will your integrated dashboard help an instructor identify which students to check-in with, and why? (5 points)

My integrated dashboard will help an instructor identify which students to check-in with by showing how probable it can be for the students to fail the instructor's course. The integrated dashboard can help show which students need more help than other students and help improve their overall scores. 

The integrated dashboard merges many capabilties into one useful tool, so by merging the capabilties into a single, functional dashboard, the instructors can identify which students to check-in with, in a more efficient manner. 

The integrated dashboard will help also instructors show which students needs to focus more on, such as quizzes or tests or both; as well as focus on which course material needs to be taught more in detail and which other course materials needs less attention.

2.2 Based on the readings and videos, what would you recommend as improvements to your dashboard, and why? (5 points)

Based on the readings and videos, I would recommend setting a feauture to the dashboard which sends out emails and/or texts to the students to remind them to check the dashboard daily or weekly.  

I would also recommend a feature in which the students also have input, so that the instructors can see the student's, or students', POV, as an improvement to my dashboard. 